---
**License**

 VexpBounce

 Tue Nov 15 12:13:14 2022\
 Copyright  2022\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 MagDustBounce\
 Copyright (C) 2022 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import cm

from numcosmo_py import Ncm, Nc
from numcosmo_py.plotting.tools import set_rc_params_article, latex_float

# Initializing the library

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

# Latex definition

In [ ]:
set_rc_params_article(ncol=2)

# Model initialization

Creating a new background Vexp model

In [ ]:
Vexp = Nc.HICosmoVexp.new()

current_set = {
    "alphab": 7.4847e-3,  # Alpha (# of e-fold\s) at the bounce
    "sigmaphi": 100.0,  # Width of the Gaussian solution for the WdW equation
    "xb": 2.0e36,  # Inverse of the scale factor at the bounce (Initial condition)
    "dphi": -9.0e-4,  # Deviation of the Gaussian solution for the WdW equation
    "OmegaL": 1.0,  # H²(a when w=-1)/H²(a0). Basically gives the DE-dominated phase
    "Omegac": 1.0,  # Omega_d???
    "H0": 67.8,  # Hubble parameter today given by CMB observations
    "Bem": 1.0e60,  # Amplitude of the EM gaussian coupling
    "betaem": 3.2,  # Width of the EM gaussian coupling
}

Vexp.set_properties(**current_set)

tc = Vexp.tau_xe(
    1.0e15
)  # Initial tau in expansion phase??? tau para quando a expansão é 10^15
reltol = 1.0e-12  # Relative tolerance for solving differential equations

Vexp.tau_xe(1.0)

# Model integrated time

Getting the time intervals for the computed model in terms of $\alpha \equiv \ln(a/a_0)$ and $\alpha = \alpha_b + \frac{\tau^2}{2}$.



In [ ]:
tau_min = Vexp.tau_min()  # Get initial time
tau_max = Vexp.tau_max()  # Get final time
tau_qt_c = Vexp.tau_qt_c()  # Get initial time of the quantum phase
tau_qt_e = Vexp.tau_qt_e()  # Get final time of the quantum phase
npoints = 5000

tau_a = np.linspace(tau_min, tau_max, npoints)
tau_q_a = np.linspace(tau_qt_c, tau_qt_e, npoints)

print(f"Vexp model prepared in the interval ({tau_min}, {tau_max})")
print(f"Vexp quantum dominated interval ({tau_qt_c}, {tau_qt_e})")

Computing background functions

In [ ]:
alpha_a = [Vexp.alpha(tau) for tau in tau_a]
phi_a = np.array([Vexp.phi(tau) for tau in tau_a])
phi_q_a = np.array([Vexp.phi(tau) for tau in tau_q_a])
deta_s = Ncm.SplineCubicNotaknot.new()
deta_s.set_array(tau_a, [Vexp.eval_nu(tau, 1.0) for tau in tau_a], True)
rescale = Vexp.RH_planck() / np.exp(Vexp.alpha_0e())
eta_a = [
    (
        -deta_s.eval_integ(tau, 0.0) * rescale
        if tau < 1.0
        else deta_s.eval_integ(0.0, tau) * rescale
    )
    for tau in tau_a
]
RS_a = np.array(
    [Vexp.Ricci_scale(tau) for tau in tau_a]
)  # Hubble scale goes to 0 at the bounce, but not the Ricci scalar
nu_a = np.array([Vexp.eval_nu(tau, 1.0) for tau in tau_a])
x_y_a = np.array([Vexp.x_y(tau) for tau in tau_a])

# Consistence check

Plotting $\alpha(\tau) - \alpha_b$.

In [ ]:
plt.figure()

plt.plot(tau_a, alpha_a, label=r"$\alpha$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.show()
pass

# Plotting scale factor as a function of tau.

$a(\tau)$

In [ ]:
plt.figure()

print(Vexp.alpha_0e())  # alpha_0 na expansão
print(Vexp.tau_xe(1.0))

plt.plot(tau_a, np.exp(alpha_a) / Vexp.props.xb, label=r"$a$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.yscale("log")
plt.show()
pass

# Plotting the scalar field as a function of tau.

$\phi(\tau)$

In [ ]:
plt.figure()

plt.plot(tau_a, phi_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.show()

In [ ]:
plt.figure()

plt.plot(tau_q_a, phi_q_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.show()

In [ ]:
plt.figure()

plt.plot(eta_a, phi_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xscale("symlog", linthresh=1.0e-1)
plt.xlabel(r"$\eta$")
plt.show()

# Ploting Ricci scalar and mode frequency.

$R(\tau)$ and $\nu(\tau)$

In [ ]:
plt.figure()

plt.plot(tau_a, RS_a, label=r"$R_s$")
plt.plot(tau_a, nu_a, label=r"$\nu$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.yscale("log")

plt.show()

# Plotting background field variables

$x(\tau)$, $y(\tau)$ and $\sqrt{x^2 + y^2}$.

In [ ]:
plt.figure()

plt.plot(tau_a, x_y_a[:, 0], label="$x$")
plt.plot(tau_a, x_y_a[:, 1], label="$y$")
plt.plot(tau_a, np.hypot(x_y_a[:, 0], x_y_a[:, 1]), label=r"$\sqrt{x^2+y^2}$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.yscale("symlog")

plt.show()

# Close-up on the Friedmann constraint in the quantum regime
plt.figure()

plt.plot(
    tau_a,
    np.sqrt(x_y_a[:, 0] * x_y_a[:, 0] + x_y_a[:, 1] * x_y_a[:, 1]),
    label="$x^2+y^2$",
)
plt.plot(
    tau_a,
    np.hypot(x_y_a[:, 0], x_y_a[:, 1]),
    label=r"$\sqrt{x^2+y^2}$",
    linestyle="dashed",
)

plt.xlim(-0.25, 0.25)
plt.yscale("symlog")

plt.show()

# Plotting magnetic field background quantities 

$\phi(\tau)$, $f(\phi(\tau)) = e^{-\phi^2/\beta^2}$ and the equation of state $w$.

In [ ]:
plt.figure()

plt.plot(tau_a, phi_a, label=r"$\phi$")
# plt.plot (tau_a, np.exp (-(phi_a / beta)**2), label=r'$f(\phi)$')
plt.plot(tau_a, 2.0 * x_y_a[:, 0] ** 2 - 1.0, label="$w = 2x^2-1$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
# plt.yscale('symlog')
plt.xlim(-2.0, tau_qt_e)
plt.ylim(-1.1, 1.1)

plt.show()

###
### Equation of state
###

plt.figure()

plt.plot(tau_a, 2.0 * x_y_a[:, 0] ** 2 - 1.0, label="$w = 2x^2-1$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.xlim(-0.025, 0.025)
plt.xlabel(r"$\tau$")

plt.show()

# Defines the electromagnetic field model

This model use the Complex Structure Quantization method CSQ1D.

In [ ]:
tau_f = 9.3

pad = Nc.HIPertAdiab.new()
pad.set_initial_condition_type(Ncm.CSQ1DInitialStateType.ADIABATIC4)
pad.set_k(1.0)
pad.set_ti(Vexp.tau_min())
pad.set_tf(tau_f)
pad.set_vacuum_max_time(-1.0)
pad.set_vacuum_reltol(1.0e-8)
pad.set_reltol(reltol)

# Finding starting point as the end of the adiabatic phase

Setting limit for the adiabatic phase with $10^{-9}$.

In [ ]:
print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

(Found_tau_i, tau_i) = pad.find_adiab_time_limit(
    Vexp, tau_min, -12.0, 1.0e-9
)
print(f"{Found_tau_i} {tau_i}")

# Checking if the search for the starting time was successful 

In [ ]:
print(
    f"Found ({Found_tau_i}) WKB scale for no-potential at     {tau_i:7.3f}"
)

print(pad.eval_F1(Vexp, tau_i))
print(pad.eval_F2(Vexp, tau_i))

state = Ncm.CSQ1DState.new()

print(pad.compute_adiab(Vexp, tau_i, state)[0].get_ag())

In [ ]:
plt.figure()

tau_ad_a = tau_a

F1_a = np.abs([pad.eval_F1(Vexp, tau) for tau in tau_ad_a])
F2_a = np.abs([pad.eval_F2(Vexp, tau) for tau in tau_ad_a])

plt.plot(tau_ad_a, F1_a, label=r"$|F_1|$")
plt.plot(tau_ad_a, F2_a, label=r"$|F_2|$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.ylim(1.0e-20, None)
plt.xlabel(r"$\tau$")
plt.yscale("log")

plt.show()

In [ ]:
pad.prepare(Vexp)
tau_evol = np.array(pad.get_time_array()[0])
Pzeta_array = [pad.eval_powspec_zeta_at(Vexp, tau) for tau in tau_evol]


In [ ]:
fig1, ax1 = plt.subplots()
mylw=1
plt.plot(tau_evol, Pzeta_array, lw=mylw, label=fr"$P_\zeta(k_s = {pad.get_k()})$")

ax1.legend(loc="best", ncol=2, fontsize=13)
ax1.set_yscale("log")

In [ ]:
tau_ad_a = np.linspace(-6.0, 6.00, 6)

ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 300)
k_a_plot = np.geomspace(ki, kf, 1000)

pad.prepare_spectrum(Vexp, k_a, tau_ad_a)

In [ ]:
Pzeta = pad.eval_powspec_zeta(Vexp)

In [ ]:
tau = 5.0
ps_zeta_tau = [Pzeta.eval(Vexp, tau, k) for k in k_a_plot]
coeffs = np.polyfit(np.log(k_a_plot), np.log(k_a_plot**3 * ps_zeta_tau), 1)

In [ ]:
fig1, ax1 = plt.subplots()
mylw=1
plt.plot(k_a_plot, k_a_plot**3 * ps_zeta_tau, lw=mylw, label=fr"$k^3P_\zeta(\tau = {tau})$")
plt.plot(k_a_plot, np.exp(np.log(k_a_plot) * coeffs[0] + coeffs[1]), lw=mylw, label=fr"linear fit")

ax1.legend(loc="best", ncol=2, fontsize=13)
ax1.set_xscale("log")
ax1.set_yscale("log")

In [ ]:
coeffs[0]